## Parallel sorting


### Sorting network - bitonic sorting algorithm

- Bitonic Sequence
    - A sequence is called Bitonic if it is first increasing, then decreasing. In other words, an array arr[0..n-i] is Bitonic if there exists an index i where 0<=i<=n-1 such that $$ x0 <= x1 …..<= xi  \text{ and }  xi >= xi+1….. >= xn-1 $$
    
  <img src="images/Batcher_Bitonic_Mergesort_for_eight_inputs.svg.png" width=600 />

In [306]:
%%writefile BitonicSort.java
import java.util.concurrent.CyclicBarrier;

public class BitonicSort {
    
    static final int width = 4; // counting network width
    static final int depth = 8; // counting network depth
    static final int p = 4; // number of threads
    static final int s = 1; // p*s is a power of 2
    static final int[][][] bitonicTable = new int[p*s*depth][depth][2];

    static CyclicBarrier barrier = new CyclicBarrier(p);
    static int test_data[]= {3, 7, 4, 8, 6, 2, 1, 5};
    
    /*
    public void generateTable(int[][][] bitonicTable){
        for(int d=0; d<depth; d++){
            for(int j=0; j<s; j++){
                for(int i)
                if(d%2==0){
                    bitonicTable[][d][0]=
                    bitonicTable[(i*s)+j][d][1]=
            }
        }
        
    }
    */
    
    public static void sort(int[] items) {
        int i = (int) Thread.currentThread().getId();
        for (int d = 0; d < depth; d++) {
            try{barrier.await();} catch(Exception e){}
            for (int j = 0; j < s; j++) {
                int north = bitonicTable[(i*s)+j][d][0];
                int south = bitonicTable[(i*s)+j][d][1];
                //System.out.println(north);
                //System.out.println(south);
                //System.out.println("----");
                if (items[north] < items[south]) {
                    System.out.println(items[north] + ','+items[south]);
                    int temp = items[north];
                    items[north] = items[south];
                    items[south] = temp;
                    System.out.println(items[north] + ','+items[south]);
                }
            }
        }
    } 
    
    static Thread thread(int id) {
        return new Thread(() -> {
            sort(test_data);
        });
    }
    
    public static void main(String[] args){
        
        System.out.println("input:");
        for(int x : test_data){
            System.out.print(x);
            System.out.print(' ');
        }
        System.out.println("\n");
        Thread[] t = new Thread[p];
        long start = System.currentTimeMillis();
        for (int i=0; i<p; i++)
          t[i] = thread(i);
        for (int i=0; i<p; i++)
          t[i].start();
        try {
        for (int i=0; i<p; i++)
          t[i].join();
        } catch (InterruptedException e) {}
        long stop = System.currentTimeMillis();
        
        System.out.println("output:");
        for(int x : test_data){
            System.out.print(x);
            System.out.print(' ');
        }
        System.out.println("\ntime: "+(stop-start)+" ms");
        
    }
    
}

Overwriting BitonicSort.java


In [307]:
!javac BitonicSort.java

In [308]:
!java -ea BitonicSort

input:
3 7 4 8 6 2 1 5 

output:
3 7 4 8 6 2 1 5 
time: 9 ms


The BitonicSort takes $O(s log^2 p)$ time for p threads running on p processors, which, if s is constant, is $O(log^2 p)$ time.